In [1]:
import tensorflow as tf
import os
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
PATH_DRIVE = "/content/gdrive/My Drive/CV_project/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Ngrock Package to see results in Tensorboard WHEN USING GOOGLE COLAB

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

LOG_DIR = './Graph'
get_ipython().system_raw(
    'yes | tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


--2018-11-20 00:11:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.22.34.127, 52.44.144.199, 52.55.191.55, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.22.34.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  17.5MB/s    in 0.3s    

2018-11-20 00:11:05 (17.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
model_dir = './Graph'
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

In [5]:
captions_zip = tf.keras.utils.get_file('Flickr8k_text.zip', 
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_text.zip',
                                          extract = True)
captions_token = os.path.dirname(captions_zip)+'/Flickr8k.token.txt'
captions_trainImages = os.path.dirname(captions_zip)+'/Flickr_8k.trainImages.txt'
captions_devImages = os.path.dirname(captions_zip)+'/Flickr_8k.devImages.txt'

2342912/2340801 [==============================] - 0s 0us/step


In [0]:
# Path to save the results in Google drive
path_features_vgg = PATH_DRIVE + 'features_vgg.pkl'
path_features_our = PATH_DRIVE + 'features_our.pkl'
path_captions = PATH_DRIVE + 'captions.txt'

In [0]:
# read document file from their path :

with open(captions_trainImages, 'r') as f:
  doc_trainImages = f.read()

with open(path_captions, 'r') as f:
  doc_captions = f.read()

In [8]:
# convert the content to a dataset
dataset = list()
for line in doc_trainImages.split('\n'):
  if len(line) < 1:
    continue
  identifier = line.split('.')[0]
  dataset.append(identifier)
train = set(dataset)

# print the size of our dataset
print('Dataset size: %d' % len(train))

Dataset size: 6000


In [9]:
# for each id in the train image folder, 
# a cleaned caption is acssociated
train_captions = dict()
for line in doc_captions.split('\n'):
  tokens = line.split()
  image_id, image_desc = tokens[0], tokens[1:]

  if image_id in train:
    if image_id not in train_captions:
      train_captions[image_id] = list()
    desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
    train_captions[image_id].append(desc)

# print the size of our training captions set
print('Captions train size : %d' % len(train_captions))

Captions train size : 6000


In [14]:
#NATURAL LANGUAGE PROCESSING PART adapted from onlie code because not seen in class

#convert the dictionnary to list
lines = list()
for key in train_captions.keys():
  [lines.append(d) for d in train_captions[key]]

#create a tokenizer from the list
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)

vocab_size = (len(tokenizer.word_index) + 1)
print('Vocabulary Size: %d' % vocab_size)

max_length = max(len(d.split()) for d in lines)
print('Caption Length: %d' % max_length)

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, capt_list, photo):
  photo_save, start_cap_save, end_cap_save = list(), list(), list()
  
  # for every image
  for capt in capt_list:
    seq = tokenizer.texts_to_sequences([capt])[0]
    for i in range(1, len(seq)):
      start_cap, end_cap = seq[:i], seq[i]
      start_cap = pad_sequences([start_cap], maxlen=max_length)[0]
      end_cap = to_categorical([end_cap], num_classes=vocab_size)[0]
			
      # save it to local variables
      photo_save.append(photo)
      start_cap_save.append(start_cap)
      end_cap_save.append(end_cap)
  return array(photo_save), array(start_cap_save), array(end_cap_save)

# function to create a generator throuht the yield function
# have been created in order to call model.fit_generator() later on
def data_generator(descriptions, photos, tokenizer, max_length):
  while 1:
    for key, desc_list in descriptions.items():
      photo = photos[key][0]
      # image = np.reshape(image, (4096,))
      in_img, start_cap, out_word = create_sequences(tokenizer, max_length, desc_list, photo)
      
      yield [[in_img, start_cap], out_word]

Vocabulary Size: 7579
Caption Length: 34


In [0]:
# define the captioning model
def define_model(vocab_size, max_length):
    
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
  
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(256)(se2)
  
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
  
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam')
  
	return model

In [12]:
# Get the features from the images for each of our model 

features_vgg = load(open(path_features_vgg, 'rb'))
train_features_vgg = {k: features_vgg[k] for k in train}
print('Photos train vgg model, size : %d' % len(train_features_vgg))

features_our = load(open(path_features_our, 'rb'))
train_features_our = {k: features_our[k] for k in train}
print('Photos: train our model, size :%d' % len(train_features_our))

Photos train vgg model, size : 6000
Photos: train our model, size :6000


In [15]:
model = define_model(vocab_size, max_length)
epochs = 4
steps = len(train_captions)
for i in range(epochs):
	generator = data_generator(train_captions, train_features_our, tokenizer, max_length)
	model.fit_generator(generator, epochs=1, steps_per_epoch=steps,callbacks=[tbCallBack], verbose=1)
	model.save(PATH_DRIVE+'model_our_' + str(i) + '.h5')

Epoch 1/1
 289/6000 [>.............................] - ETA: 15:04 - loss: 6.0145

KeyboardInterrupt: ignored

In [0]:
model = define_model(vocab_size, max_length)
epochs = 4
steps = len(train_captions)
for i in range(epochs):
	generator = data_generator(train_captions, train_features_vgg, tokenizer, max_length)
	model.fit_generator(generator, epochs=1, steps_per_epoch=steps,callbacks=[tbCallBack], verbose=1)
	model.save(PATH_DRIVE+'model_vgg_' + str(i) + '.h5')

In [0]:
# Link to use to see Tensorboard WHEN USING GOOGLE COLAB

get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"